In [1]:
print("test")

test


In [44]:
import numpy as np
theta = 30
img = np.random.randint(0, 255, size=(9, 6))
img_rows, img_cols = img.shape
# 计算图像对角线长度（投影的最大范围）
diag_length = int(np.ceil(np.sqrt(img_rows**2 + img_cols**2)))
projection = np.zeros(diag_length)
theta_rad = np.deg2rad(theta)
half_col = img_cols // 2
half_row = img_rows // 2
x = np.arange(-half_col, img_cols - half_col)
x_centers = np.arange(img_cols) - (img_cols - 1) / 2.0
y = np.arange(-half_row, img_rows - half_row)
X, Y = np.meshgrid(x, y)
X_rotat = X * np.cos(theta_rad) - Y * np.sin(theta_rad)
x_centers

array([-2.5, -1.5, -0.5,  0.5,  1.5,  2.5])

In [39]:
img

array([[ 70, 144, 221,   9,   5, 105],
       [111, 243, 188,  68, 197, 232],
       [138,  37, 171, 226, 149, 231],
       [ 24,  93, 106,  91, 176, 251],
       [ 32, 143, 247,  68, 146,  28],
       [ 55, 125, 236, 254,  21,  37],
       [241, 195, 211,  66, 242,  24],
       [139,  33, 197, 117, 184, 208],
       [ 32, 132,  52, 224, 155,   7]])

In [42]:
np.zeros(10)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [43]:
import numpy as np

def parallel_projection(img, theta_deg, width=1.0):
    """
    近似平行投影
    - img: 2D 灰度图像（numpy array）
    - theta_deg: 投影角度，度
    - width: 条带宽（像素），默认 1.0，相当于 half=0.5（半宽）
    返回:
      projection: 1D array，长度 = diag_length (整型)
    """
    img_rows, img_cols = img.shape
    x_centers = np.arange(img_cols) - (img_cols - 1) / 2.0
    y_centers = np.arange(img_rows) - (img_rows - 1) / 2.0
    # X[r,c] 给出像素 (r,c) 的 x 坐标；Y[r,c] 给出像素 (r,c) 的 y 坐标
    X, Y = np.meshgrid(x_centers, y_centers)   # both shape (img_rows, img_cols)
    diag_length = int(np.ceil(np.hypot(img_rows, img_cols)))
    s_coords = np.linspace(-(diag_length - 1) / 2.0, (diag_length - 1) / 2.0, diag_length)
    theta = np.deg2rad(theta_deg)
    # X_rot 表示每个像素中心在投影轴上的坐标 s = x*cosθ - y*sinθ, cuz y 轴是向下的
    X_rot = X * np.cos(theta) - Y * np.sin(theta)

    half = width / 2.0
    projection = np.zeros_like(s_coords)
    for i, s in enumerate(s_coords):
        # mask 表示哪些像素的投影坐标落在 [s-half, s+half)
        mask = np.abs(X_rot - s) <= half
        # 累加被选像素的强度
        projection[i] = np.sum(img[mask])

    return projection

In [4]:
import numpy as np

def backproject(sinogram, thetas, output_shape, width=1.0):
    """
    简单反投影函数，将sinogram反投影为图像
    参数:
        sinogram: 正弦图，形状为(num_angles, diag_length)
        thetas: 投影角度列表(度)，与sinogram的行对应
        output_shape: 输出图像的形状 (rows, cols)
        width: 条带宽，需与正投影时保持一致，默认1.0
    返回:
        recon: 反投影重建的图像
    """
    img_rows, img_cols = output_shape
    num_angles, diag_length = sinogram.shape

    # 初始化重建图像
    recon = np.zeros(output_shape, dtype=np.float64)

    # 计算像素中心相对于图像中心的坐标
    x_centers = np.arange(img_cols) - (img_cols - 1) / 2.0
    y_centers = np.arange(img_rows) - (img_rows - 1) / 2.0
    X, Y = np.meshgrid(x_centers, y_centers)  # 形状均为(img_rows, img_cols)

    # 生成投影轴s的坐标（与正投影保持一致）
    s_coords = np.linspace(-(diag_length - 1) / 2.0,
                          (diag_length - 1) / 2.0,
                          diag_length)
    s0 = s_coords[0]  # s坐标的起始值
    half = width / 2.0

    # 对每个角度进行反投影
    for i_theta in range(num_angles):
        theta_deg = thetas[i_theta]
        theta = np.deg2rad(theta_deg)

        # 计算该角度下每个像素的投影坐标s = x*cosθ - y*sinθ
        X_rot = X * np.cos(theta) - Y * np.sin(theta)

        # 计算每个像素对应的投影索引
        i_s = np.round(X_rot - s0).astype(int)
        # 确保索引在有效范围内
        i_s = np.clip(i_s, 0, diag_length - 1)

        # 获取当前角度的投影值并反投影到图像
        projection = sinogram[i_theta]
        recon += projection[i_s] # 等价于对每个像素做 recon[r,c] += projection[i_s[r,c]]

    return recon

In [5]:
import cv2
def load_image(path):
    """读取图像并转为灰度图"""
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise FileNotFoundError(f"无法读取图像: {path}")

    return img

def parallel_projection(img, theta_deg, width=1.0):
    """
    近似平行投影
    - img: 2D 灰度图像（numpy array）
    - theta_deg: 投影角度，度
    - width: 条带宽（像素），默认 1.0，相当于 half=0.5（半宽）
    返回:
      projection: 1D array，长度 = diag_length (整型)
    """
    img_rows, img_cols = img.shape
    x_centers = np.arange(img_cols) - (img_cols - 1) / 2.0
    y_centers = np.arange(img_rows) - (img_rows - 1) / 2.0
    # X[r,c] 给出像素 (r,c) 的 x 坐标；Y[r,c] 给出像素 (r,c) 的 y 坐标
    X, Y = np.meshgrid(x_centers, y_centers)   # both shape (img_rows, img_cols)
    diag_length = int(np.ceil(np.hypot(img_rows, img_cols)))
    s_coords = np.linspace(-(diag_length - 1) / 2.0, (diag_length - 1) / 2.0, diag_length)
    theta = np.deg2rad(theta_deg)
    # X_rot 表示每个像素中心在投影轴上的坐标 s = x*cosθ - y*sinθ, cuz y 轴是向下的
    X_rot = X * np.cos(theta) - Y * np.sin(theta)

    half = width / 2.0
    projection = np.zeros_like(s_coords)
    for i, s in enumerate(s_coords):
        # mask 表示哪些像素的投影坐标落在 [s-half, s+half)
        mask = np.abs(X_rot - s) <= half
        # 累加被选像素的强度
        projection[i] = np.sum(img[mask])

    return projection

def generate_sinogram(img, thetas, width=1.0):
    """
    对一组角度生成 sinogram
    返回数组形状: (num_angles, diag_length)
    """
    projs = [parallel_projection(img, th, width=width) for th in thetas]
    sinogram = np.vstack(projs)  # 每行对应一个角度
    return sinogram



In [6]:
img_size = 128
im = load_image('phantom.bmp')
imf = im.astype(np.float32) / 255.0
img = cv2.resize(imf, (img_size, img_size), interpolation=cv2.INTER_AREA)
sinogram = generate_sinogram(img, thetas=np.arange(0, 180, 1), width=1.0)
recon = backproject(sinogram, thetas=np.arange(0, 180, 1), output_shape=img.shape, width=1.0)

In [7]:
import numpy as np
recon_freq = np.fft.fft2(recon)